In [ ]:
import netsquid as ns
# Set qstate formalism to use density matrices.
ns.set_qstate_formalism(ns.QFormalism.DM)

# Exercise
**Starting Point:** Tutorial 3 example code

## Verify Notebook
Run the simulation in this notebook to ensure your notebook environment is configured correctly.
Run the simulation 5-10 times, do Alice and Bob always measure the same result?

## Change Noise Model
Replace the current quantum noise model for the quantumchannels with a `DephaseNoiseModel`.
Run the simulation again and note the difference in density matrix.
Can Alice and Bob measure different results on their qubits with this noise model?

## Create Entanglement Using Quantum Gates
Create entanglment between qubits q1 and q2 by applying quantum operators instead of directly assigning a quantum state.
**Current approach to remove:**
```python
ns.qubits.assign_qstate([q1, q2], ns.b00)
```
**New approach to implement:**pp
- Create the Bell state |Φ⁺⟩ = (1/√2)(|00⟩ + |11⟩) using quantum operators
- Apply the following gate sequence:
 1. Apply Hadamard gate (H) to the first qubit (q1)
 2. Apply CNOT gate with q1 as control and q2 as target
**Quantum Circuit:**
```
q1: ──H──●──
         │
q2: ─────X──
```
## Finished?
For a larger challenge try the [extra exercise](extra1.ipynb).


# Exercise starting point:

## Network setup

In [ ]:
from netsquid.nodes import Node
from netsquid.components import Port, QuantumChannel
from netsquid.nodes import DirectConnection
from netsquid.components.models import FixedDelayModel, DepolarNoiseModel

alice_node = Node("Alice")
bob_node   = Node("Bob")

alice_port = Port("Alice_port", alice_node)
bob_port   = Port("Bob_port",   bob_node)

delay_model = FixedDelayModel(delay=10.)
error_model = DepolarNoiseModel(depolar_rate=0.2, time_independent=True)

channel_a_to_b = QuantumChannel("Channel Alice to Bob",
                                  models={"delay_model": delay_model,
                                          "quantum_noise_model": error_model}
                                  )

channel_b_to_a = QuantumChannel("Channel Bob to Alice",
                                  models={"delay_model": delay_model,
                                          "quantum_noise_model": error_model}
                                  )


connection = DirectConnection(name="connection",
                              channel_AtoB=channel_a_to_b,
                              channel_BtoA=channel_b_to_a)


alice_node.connect_to(remote_node=bob_node, connection=connection,
                     local_port_name="Alice_port", remote_port_name="Bob_port")

## Protocols

In [ ]:
from netsquid.protocols import NodeProtocol

class AliceProtocol(NodeProtocol):
    def run(self):
        port = self.node.ports["Alice_port"]

        q1, q2 = ns.qubits.create_qubits(2)
        
        ns.qubits.assign_qstate([q1, q2], ns.b00)
        
        port.tx_output(q2)
        print(f"{ns.sim_time()} ns Alice send q2")

        # Wait 30 ns
        yield self.await_timer(30)

        m, _ = ns.qubits.measure(q1)

        print(f"{ns.sim_time()} ns Alice measures q1: {m}")



class BobProtocol(NodeProtocol):
    def run(self):
        port = self.node.ports["Bob_port"]

        yield self.await_port_input(port)
        
        q2 = port.rx_input().items[0]

        dm = ns.qubits.reduced_dm(q2.qstate.qubits)
        print(f"{ns.sim_time()} ns Bob receives q2, DM=\n{dm}")

        m, _ = ns.qubits.measure(q2)

        print(f"{ns.sim_time()} ns Bob measures q2: {m}")

alice_protocol = AliceProtocol(node=alice_node)
bob_protocol = BobProtocol(node=bob_node)

## Run simulation

In [ ]:
# Start all protocols
alice_protocol.start()
bob_protocol.start()

sim_stats = ns.sim_run()

print(sim_stats)

# Reset simulation
alice_protocol.stop()
bob_protocol.stop()

ns.sim_reset()